In [1]:
from keras.datasets import cifar10

(x_train_and_validation, y_train_and_validation), (x_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


In [2]:
x_train_and_validation.shape

(50000, 32, 32, 3)

In [3]:
import matplotlib.pyplot as plt

single_img = x_train_and_validation[0]

plt.imshow(single_img)

In [4]:
print(x_train_and_validation.max(), x_train_and_validation.min(), sep='\n')

255
0


In [5]:
scaled_x_train_and_validation = x_train_and_validation/255

scaled_x_test = x_test/255

print(scaled_x_test.max(),scaled_x_test.min(),sep='\n')

1.0
0.0


In [6]:
import numpy as np

scaled_x_train_and_validation.shape[0]

50000

In [7]:
validation_x_train_num = np.int(scaled_x_train_and_validation.shape[0]*0.1)

scaled_x_validation = scaled_x_train_and_validation[:validation_x_train_num]
scaled_x_train = scaled_x_train_and_validation[validation_x_train_num:]

y_validation = y_train_and_validation[:validation_x_train_num]
y_train = y_train_and_validation[validation_x_train_num:]

print(scaled_x_validation.shape[0], scaled_x_train.shape[0], y_validation.shape[0], y_train.shape[0])

5000 45000 5000 45000


In [8]:
from keras import callbacks
from keras.layers import Conv2D,Dense,Flatten,MaxPool2D
from keras.models import Sequential

In [9]:
model = Sequential()

## FIRST SET OF LAYERS

# CONVOLUTIONAL LAYER
model.add(Conv2D(filters=32, kernel_size=(4,4),input_shape=(32, 32, 3), activation='relu',))
# CONVOLUTIONAL LAYER
model.add(Conv2D(filters=32, kernel_size=(4,4),input_shape=(32, 32, 3), activation='relu',))

# POOLING LAYER
model.add(MaxPool2D(pool_size=(2, 2)))

## SECOND SET OF LAYERS

# CONVOLUTIONAL LAYER
model.add(Conv2D(filters=64, kernel_size=(4,4),input_shape=(32, 32, 3), activation='relu',))
# CONVOLUTIONAL LAYER
model.add(Conv2D(filters=64, kernel_size=(4,4),input_shape=(32, 32, 3), activation='relu',))

# POOLING LAYER
model.add(MaxPool2D(pool_size=(2, 2)))

# FLATTEN IMAGES BEFORE FINAL LAYER
model.add(Flatten())

# 512 NEURONS IN DENSE HIDDEN LAYER
model.add(Dense(512, activation='relu'))

# LAST LAYER IS THE CLASSIFIER, THUS 10 POSSIBLE CLASSES
model.add(Dense(10, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 29, 29, 32)        1568      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        16416     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        32832     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 64)          65600     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)              

In [11]:
BATCH_SIZE = 20
MAX_EPOCHS = 20
EARLY_STOPPING = callbacks.EarlyStopping(patience=2)

model.fit(scaled_x_train,
          y_train,
          epochs=MAX_EPOCHS,
          batch_size = BATCH_SIZE,
          validation_data=(scaled_x_validation, y_validation),
          callbacks = [EARLY_STOPPING],
          verbose=2)

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
 - 210s - loss: 1.5926 - accuracy: 0.4112 - val_loss: 1.3014 - val_accuracy: 0.5264
Epoch 2/20
 - 215s - loss: 1.2070 - accuracy: 0.5704 - val_loss: 1.1070 - val_accuracy: 0.6088
Epoch 3/20
 - 216s - loss: 1.0484 - accuracy: 0.6283 - val_loss: 1.0579 - val_accuracy: 0.6282
Epoch 4/20
 - 215s - loss: 0.9438 - accuracy: 0.6690 - val_loss: 0.9497 - val_accuracy: 0.6676
Epoch 5/20
 - 212s - loss: 0.8380 - accuracy: 0.7045 - val_loss: 0.8871 - val_accuracy: 0.6928
Epoch 6/20
 - 210s - loss: 0.7609 - accuracy: 0.7322 - val_loss: 0.8587 - val_accuracy: 0.6964
Epoch 7/20
 - 212s - loss: 0.6994 - accuracy: 0.7518 - val_loss: 0.8225 - val_accuracy: 0.7140
Epoch 8/20
 - 210s - loss: 0.6204 - accuracy: 0.7790 - val_loss: 0.8195 - val_accuracy: 0.7250
Epoch 9/20
 - 209s - loss: 0.5657 - accuracy: 0.7999 - val_loss: 0.8497 - val_accuracy: 0.7116
Epoch 10/20
 - 210s - loss: 0.5037 - accuracy: 0.8193 - val_loss: 0.8463 - val_accuracy: 0.7272


In [12]:
model.metrics_names

['loss', 'accuracy']

In [13]:
model.evaluate(x_test,y_test)

10000/10000 [==============================] - 11s 1ms/step


[226.88957572021485, 0.5464000105857849]

In [14]:
from sklearn.metrics import classification_report

predictions = model.predict_classes(x_test)

In [15]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.39      0.88      0.54      1000
           1       0.65      0.75      0.69      1000
           2       0.60      0.25      0.35      1000
           3       0.51      0.27      0.35      1000
           4       0.79      0.18      0.29      1000
           5       0.60      0.53      0.56      1000
           6       0.89      0.34      0.49      1000
           7       0.53      0.75      0.62      1000
           8       0.58      0.71      0.64      1000
           9       0.54      0.81      0.65      1000

    accuracy                           0.55     10000
   macro avg       0.61      0.55      0.52     10000
weighted avg       0.61      0.55      0.52     10000



In [ ]:
model.save('./models/CIFAR10_CNN_Keras.h5')